In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os,sys
from pathlib import Path
import json 
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-7s %(message)s',
                    stream=sys.stderr, level=logging.INFO)
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.INFO)

#General ML 
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, silhouette_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from itertools import combinations

#In-house Module Imports
from config import Configuration 
from datasets import EclipseSampledDataset, VoltaSampledDataset
from utils import *

In [4]:
user = "aksar"
logging.warning(f'Are you sure that you are: {user}?')

2022-02-21 00:26:49,357 WARNING Are you sure that you are: aksar?


In [5]:
#Update these if you are not the desired user
OUTPUT_DIR = f'/projectnb/peaclab-mon/{user}/unseen_experiments'
SYSTEM = 'volta'
EXP_NAME = 'tpds_experiments'
CV_INDEX = 0
FEATURE_SELECTION = False
SCALER = 'None' #For now, do the scaling inside the notebook, then you can move that to the class function
MODEL_CONFIG = 'experiment_2' #rf_tuncer or rf_tuncer_worst_case
logging.warning('Results will be generated in {}, double check please!'.format(MODEL_CONFIG))

2022-02-21 00:26:49,389 WARNING Results will be generated in experiment_2, double check please!


In [6]:
conf = Configuration(ipython=True,
                     overrides={
                         'output_dir': Path(OUTPUT_DIR), #change
                         'system' : SYSTEM,
                         'exp_name':EXP_NAME,                                                  
                         'cv_fold':CV_INDEX, 
                         'model_config': MODEL_CONFIG,
                     })

with open(str(conf['experiment_dir']) + '/anom_dict.json') as f:
    ANOM_DICT = json.load(f)
with open(str(conf['experiment_dir']) + '/app_dict.json') as f:
    APP_DICT = json.load(f)    
    
APP_REVERSE_DICT = {}
for app_name, app_encoding in APP_DICT.items():
    APP_REVERSE_DICT[app_encoding] = app_name    

ANOM_REVERSE_DICT = {}
for anom_name, anom_encoding in ANOM_DICT.items():
    ANOM_REVERSE_DICT[anom_encoding] = anom_name        

2022-02-21 00:26:49,419 WARNING You set windowing False, but you are trying to define window_size parameter, it is automatically set to 0. Please double check.
2022-02-21 00:26:49,420 INFO    Setting directory names
2022-02-21 00:26:49,426 INFO    Model config folder already exists, be careful, otherwise it will overwrite!
2022-02-21 00:26:49,429 INFO    Saving configuration as CSV


# The configuration used for this run:
# {'cv_fold': 0,
#  'exp_name': 'tpds_experiments',
#  'experiment_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments'),
#  'feature_extract': False,
#  'feature_select': False,
#  'hdf_data_path': PosixPath('/projectnb/peaclab-mon/aksar/datasets/tpds_data_hdfs'),
#  'metadata_path': None,
#  'model_config': 'experiment_2',
#  'model_config_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_2'),
#  'model_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_2/model'),
#  'num_split': 5,
#  'operation': 'read',
#  'output_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta'),
#  'plots_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_2/model/plots'),
#  'processed_ldms_data_path': None,
#  'raw_ldms_data_path': None,
#  'results_dir': Po

In [7]:
if SYSTEM == 'eclipse':
    
    eclipseDataset = EclipseSampledDataset(conf)
    train_data, train_label, test_data, test_label = eclipseDataset.load_dataset(scaler=SCALER,borghesi=False,tsfresh = True)

elif SYSTEM == 'volta':
    voltaDataset = VoltaSampledDataset(conf)
    train_data, train_label, test_data, test_label = voltaDataset.load_dataset(scaler=SCALER,borghesi=False)
    
assert list(train_data.index) == list(train_label.index) #check the order of the labels     
assert list(test_data.index) == list(test_label.index) #check the order of the labels    

if FEATURE_SELECTION:
    selected_features = pd.read_csv(conf['experiment_dir'] / 'selected_features.csv')
    train_data = train_data[list(selected_features['0'].values)]
    test_data = test_data[list(selected_features['0'].values)]
    
train_label['anom_names'] = train_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
train_label['app_names']=train_label['app'].apply(lambda x: APP_REVERSE_DICT[x])
test_label['anom_names'] = test_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
test_label['app_names']=test_label['app'].apply(lambda x: APP_REVERSE_DICT[x])

all_data = pd.concat([train_data, test_data])
all_data = all_data.dropna(axis=1, how='any')
all_label = pd.concat([train_label,test_label])

train_data = all_data.loc[train_label.index]
test_data = all_data.loc[test_label.index]
    
logging.info("Train data shape %s",train_data.shape)
logging.info("Train label shape %s",train_label.shape)
logging.info("Test data shape %s",test_data.shape)  
logging.info("Test label shape %s",test_label.shape)

logging.info("Train data label dist: \n%s",train_label['anom'].value_counts())
logging.info("Test data label dist: \n%s",test_label['anom'].value_counts())        

2022-02-21 00:26:49,491 INFO    BaseDataset Class Initialization
2022-02-21 00:26:49,492 INFO    HPCDataset Class Initialization
2022-02-21 00:26:49,492 INFO    VoltaSampledDataset Class Initialization
2022-02-21 00:26:50,150 INFO    Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-02-21 00:26:51,861 INFO    Train data shape (14640, 1441)
2022-02-21 00:26:51,863 INFO    Train label shape (14640, 2)
2022-02-21 00:26:51,863 INFO    Test data shape (6275, 1441)
2022-02-21 00:26:51,864 INFO    Test label shape (6275, 2)
2022-02-21 00:26:51,866 WARNING Beware that no scaling method is applied
2022-02-21 00:26:52,818 INFO    Train data shape (14640, 1441)
2022-02-21 00:26:52,819 INFO    Train label shape (14640, 4)
2022-02-21 00:26:52,819 INFO    Test data shape (6275, 1441)
2022-02-21 00:26:52,820 INFO    Test label shape (6275, 4)
2022-02-21 00:26:52,826 INFO    Train data label dist: 
0    13281
2      342
1      342
3      338
4      3

## Unknown App Experiments

In [8]:
SCALER = 'MinMax'

In [9]:
nas_apps = ['lu','sp','ft','bt','cg','mg']
mantevo_apps = ['miniMD','CoMD','miniGhost','miniAMR']
other_apps = ['kripke']

all_app_names = list(APP_DICT.keys())

assert set(nas_apps + mantevo_apps + other_apps) == set(all_app_names)


cluster_one = ['ft','cg','mg']
cluster_two = ['miniAMR','lu','miniGhost']
cluster_three = ['sp','bt','miniMD','kripke','CoMD']

In [10]:
all_test_app_groups = []
all_test_app_sets = []
for app_one in combinations(cluster_one,2):
    for app_two in combinations(cluster_two,2):
        for app_three in combinations(cluster_three,2):
            all_test_app_groups.append([app_one[0],app_one[1],app_two[0],app_two[1],app_three[0],app_three[1]])
            all_test_app_sets.append(set([app_one[0],app_one[1],app_two[0],app_two[1],app_three[0],app_three[1]]))

In [11]:
#This doesn't hold the unique combinations, however it should have the total number
train_app_combinations = []

#90 different test_apps combination
for test_apps in all_test_app_groups:
    
    train_apps_all = list(set(all_app_names) - set(test_apps))
    #logging.info("Test apps: %s", test_apps)
    #logging.info("All Train apps: %s", train_apps_all)

    if len(train_apps_all) == 8:
        num_train_unknowns = [2,4,6,8]

    elif len(train_apps_all) == 5:
        num_train_unknowns = [3,5]

    else:
        raise

    #11 different train_apps combination
    for num_train_unknown in num_train_unknowns:
        for train_apps in combinations(train_apps_all, num_train_unknown):
            train_app_combinations.append(train_apps)

In [12]:
"""
This code blocks read the processed filenames so far and creates a dict using (train_apps,test_apps) as a key
It can also detect cases which are processed multiple times with different names
"""

not_covered_combinations = []
unique_counter_dict = {}
extra_filenames = []
delete_extra_files = False

for file in listdir(conf['results_dir']):
    
    if "result_dict" in file:

        filename = file.split('.')[0]
        splitted_filename = file.split('#')    

        train_apps = set(splitted_filename[0].split(':')[1].split("-"))
        test_apps = set(splitted_filename[1].split(':')[1].split("-"))
        
        if not (frozenset(train_apps),frozenset(test_apps)) in unique_counter_dict:
            unique_counter_dict[(frozenset(train_apps),frozenset(test_apps))] = 1
        else:
            unique_counter_dict[(frozenset(train_apps),frozenset(test_apps))] += 1
            extra_filenames.append(file)
                    
        if (not test_apps in all_test_app_sets) and (not train_apps in train_app_combinations):
            not_covered_combinations.append([train_apps,test_apps])
                        
sorted_unique_counter_dict = dict(sorted(unique_counter_dict.items(), key=lambda x: x[1], reverse=True))            

if delete_extra_files:
    
    for filename in extra_filenames:

        generic_filename = "#".join(filename.split("#")[:-1])

        os.remove(Path(conf['results_dir']) / filename)
        os.remove(Path(conf['results_dir']) / (generic_filename + "#alert_dict.json"))    

In [13]:
counter = 0
for test_apps in all_test_app_groups:
    
    train_apps_all = list(set(all_app_names) - set(test_apps))
    logging.info("Test apps: %s", test_apps)
    logging.info("All Train apps: %s", train_apps_all)
    
    test_apps_label = all_label[all_label['app_names'].isin(test_apps)]
    assert set(test_apps_label['app_names'].unique()) == set(test_apps)
    
    test_apps_data = all_data.loc[test_apps_label.index]
    assert list(test_apps_data.index) == list(test_apps_label.index) #check the order of the labels       
    
    if len(train_apps_all) == 8:
        num_train_unknowns = [2,4,6,8]

    elif len(train_apps_all) == 5:
        num_train_unknowns = [1,3,5]
    
    else:
        raise
            
    for num_train_unknown in num_train_unknowns:    
        
        for train_apps_selected in combinations(train_apps_all, num_train_unknown):        
            
            train_apps_selected = list(train_apps_selected)
            logging.info("Selected train apps: %s", train_apps_selected)
                        
            if not (frozenset(train_apps_selected),frozenset(test_apps)) in unique_counter_dict:                
            
                train_apps_label = all_label[all_label['app_names'].isin(train_apps_selected)]
                train_apps_data = all_data.loc[train_apps_label.index]
                assert list(train_apps_data.index) == list(train_apps_label.index) #check the order of the labels 

                if SCALER == 'MinMax':

                    scaler = MinMaxScaler()    

                elif SCALER == 'Standard':

                    scaler = StandardScaler()
                    
                clf = RandomForestClassifier(random_state=0)
                
                pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])  
                

                fit_clf = pipeline.fit(train_apps_data, train_apps_label['anom'])

                pred_test = pipeline.predict(test_apps_data)                
                logging.info("\n %s",classification_report(test_apps_label['anom'],pred_test, target_names=ANOM_DICT))
                
                result_filename = "train:" + "-".join(train_apps_selected) + "#test:" + "-".join(test_apps) \
                            + "#model:" + clf.__class__.__name__ + "#cv:" + str(CV_INDEX)                    
                logging.info(result_filename)
                result_dict = classification_report(test_apps_label['anom'],pred_test, target_names=ANOM_DICT,output_dict=True)

                with open(Path(conf['results_dir']) / (result_filename + "#result_dict.json"), "w") as fp:
                    json.dump(result_dict, fp)

                FAR_AMR_Calculate(true_label=test_apps_label['anom'],pred_label=pred_test,
                                  result_dir=conf['results_dir'],
                                  save_name=result_filename,save=True)                                   
            else: 
                counter += 1
                logging.info("File exists")

2022-02-21 00:27:04,247 INFO    Test apps: ['ft', 'cg', 'miniAMR', 'lu', 'sp', 'bt']
2022-02-21 00:27:04,248 INFO    All Train apps: ['miniGhost', 'mg', 'kripke', 'miniMD', 'CoMD']
2022-02-21 00:27:04,300 INFO    Selected train apps: ['miniGhost']
2022-02-21 00:27:05,933 INFO    
               precision    recall  f1-score   support

        none       0.99      0.11      0.21     10205
       dcopy       0.70      0.34      0.46       265
    memeater       0.02      0.92      0.05       260
        leak       0.97      0.92      0.94       255
        dial       0.88      0.12      0.21       255

    accuracy                           0.16     11240
   macro avg       0.71      0.48      0.37     11240
weighted avg       0.96      0.16      0.22     11240

2022-02-21 00:27:05,937 INFO    train:miniGhost#test:ft-cg-miniAMR-lu-sp-bt#model:RandomForestClassifier#cv:0
2022-02-21 00:27:05,960 INFO    Total misclassified normal runs: 9035, Total normal runs 10205 
2022-02-21 00:27:05,961

2022-02-21 00:27:17,541 INFO    train:mg#test:ft-cg-miniAMR-lu-sp-miniMD#model:RandomForestClassifier#cv:0
2022-02-21 00:27:17,559 INFO    Total misclassified normal runs: 7023, Total normal runs 10280 
2022-02-21 00:27:17,559 INFO    FAR: 0.683171206225681
2022-02-21 00:27:17,560 INFO    Total misclassified anom runs: 18, Total anom runs 1045 
2022-02-21 00:27:17,561 INFO    AMR: 0.01722488038277512
2022-02-21 00:27:17,563 INFO    Selected train apps: ['kripke']
/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2022-02-21 00:27:19,913 INFO    
               precision    recall  f1-score   support

        none       0.00      0.00      0.00     10280
       dcopy       0.91      0.26      0.40       26

2022-02-21 00:27:28,809 INFO    Selected train apps: ['miniMD']
2022-02-21 00:27:30,890 INFO    
               precision    recall  f1-score   support

        none       0.99      0.69      0.81     10280
       dcopy       0.99      0.94      0.97       265
    memeater       1.00      1.00      1.00       260
        leak       1.00      1.00      1.00       260
        dial       0.06      0.73      0.10       260

    accuracy                           0.71     11325
   macro avg       0.81      0.87      0.78     11325
weighted avg       0.97      0.71      0.81     11325

2022-02-21 00:27:30,890 INFO    train:miniMD#test:ft-cg-miniAMR-lu-sp-kripke#model:RandomForestClassifier#cv:0
2022-02-21 00:27:30,909 INFO    Total misclassified normal runs: 3217, Total normal runs 10280 
2022-02-21 00:27:30,909 INFO    FAR: 0.3129377431906615
2022-02-21 00:27:30,910 INFO    Total misclassified anom runs: 81, Total anom runs 1045 
2022-02-21 00:27:30,911 INFO    AMR: 0.07751196172248803
2022

2022-02-21 00:27:42,342 INFO    train:miniMD#test:ft-cg-miniAMR-lu-sp-CoMD#model:RandomForestClassifier#cv:0
2022-02-21 00:27:42,360 INFO    Total misclassified normal runs: 2936, Total normal runs 10280 
2022-02-21 00:27:42,361 INFO    FAR: 0.2856031128404669
2022-02-21 00:27:42,362 INFO    Total misclassified anom runs: 82, Total anom runs 1045 
2022-02-21 00:27:42,362 INFO    AMR: 0.0784688995215311
2022-02-21 00:27:42,364 INFO    Selected train apps: ['miniGhost', 'mg', 'kripke']
2022-02-21 00:27:42,365 INFO    File exists
2022-02-21 00:27:42,365 INFO    Selected train apps: ['miniGhost', 'mg', 'bt']
2022-02-21 00:27:42,366 INFO    File exists
2022-02-21 00:27:42,366 INFO    Selected train apps: ['miniGhost', 'mg', 'miniMD']
2022-02-21 00:27:42,367 INFO    File exists
2022-02-21 00:27:42,367 INFO    Selected train apps: ['miniGhost', 'kripke', 'bt']
2022-02-21 00:27:42,368 INFO    File exists
2022-02-21 00:27:42,368 INFO    Selected train apps: ['miniGhost', 'kripke', 'miniMD']
202

2022-02-21 00:27:51,849 INFO    File exists
2022-02-21 00:27:51,850 INFO    Selected train apps: ['sp', 'kripke', 'CoMD']
2022-02-21 00:27:51,850 INFO    File exists
2022-02-21 00:27:51,851 INFO    Selected train apps: ['mg', 'kripke', 'CoMD']
2022-02-21 00:27:51,851 INFO    File exists
2022-02-21 00:27:51,852 INFO    Selected train apps: ['miniGhost', 'sp', 'mg', 'kripke', 'CoMD']
2022-02-21 00:27:51,853 INFO    File exists
2022-02-21 00:27:51,853 INFO    Test apps: ['ft', 'cg', 'miniAMR', 'lu', 'bt', 'kripke']
2022-02-21 00:27:51,854 INFO    All Train apps: ['miniGhost', 'sp', 'mg', 'miniMD', 'CoMD']
2022-02-21 00:27:51,910 INFO    Selected train apps: ['miniGhost']
2022-02-21 00:27:53,518 INFO    
               precision    recall  f1-score   support

        none       0.98      0.26      0.41     10205
       dcopy       0.71      0.36      0.48       265
    memeater       0.03      0.75      0.05       260
        leak       0.97      0.92      0.94       255
        dial      

...

2022-02-21 00:40:42,449 INFO    File exists
2022-02-21 00:40:42,449 INFO    Selected train apps: ['sp', 'ft', 'CoMD']
2022-02-21 00:40:42,450 INFO    File exists
2022-02-21 00:40:42,450 INFO    Selected train apps: ['sp', 'ft', 'miniAMR']
2022-02-21 00:40:42,451 INFO    File exists
2022-02-21 00:40:42,452 INFO    Selected train apps: ['sp', 'CoMD', 'miniAMR']
2022-02-21 00:40:42,452 INFO    File exists
2022-02-21 00:40:42,453 INFO    Selected train apps: ['kripke', 'ft', 'CoMD']
2022-02-21 00:40:42,453 INFO    File exists
2022-02-21 00:40:42,454 INFO    Selected train apps: ['kripke', 'ft', 'miniAMR']
2022-02-21 00:40:42,454 INFO    File exists
2022-02-21 00:40:42,455 INFO    Selected train apps: ['kripke', 'CoMD', 'miniAMR']
2022-02-21 00:40:42,455 INFO    File exists
2022-02-21 00:40:42,456 INFO    Selected train apps: ['ft', 'CoMD', 'miniAMR']
2022-02-21 00:40:42,457 INFO    File exists
2022-02-21 00:40:42,457 INFO    Selected train apps: ['sp', 'kripke', 'ft', 'CoMD', 'miniAMR']
20

2022-02-21 00:40:53,922 INFO    train:sp#test:cg-mg-lu-miniGhost-bt-CoMD#model:RandomForestClassifier#cv:0
2022-02-21 00:40:53,940 INFO    Total misclassified normal runs: 5510, Total normal runs 10285 
2022-02-21 00:40:53,940 INFO    FAR: 0.5357316480311133
2022-02-21 00:40:53,941 INFO    Total misclassified anom runs: 135, Total anom runs 1055 
2022-02-21 00:40:53,942 INFO    AMR: 0.12796208530805686
2022-02-21 00:40:53,944 INFO    Selected train apps: ['kripke']
2022-02-21 00:40:56,291 INFO    
               precision    recall  f1-score   support

        none       0.98      0.06      0.11     10285
       dcopy       0.90      0.23      0.36       265
    memeater       0.03      1.00      0.05       270
        leak       0.94      0.06      0.11       260
        dial       0.00      0.00      0.00       260

    accuracy                           0.08     11340
   macro avg       0.57      0.27      0.13     11340
weighted avg       0.93      0.08      0.11     11340

2022-02

2022-02-21 00:41:08,013 INFO    train:ft#test:cg-mg-lu-miniGhost-miniMD-kripke#model:RandomForestClassifier#cv:0
2022-02-21 00:41:08,031 INFO    Total misclassified normal runs: 7020, Total normal runs 10360 
2022-02-21 00:41:08,032 INFO    FAR: 0.6776061776061776
2022-02-21 00:41:08,032 INFO    Total misclassified anom runs: 2, Total anom runs 1065 
2022-02-21 00:41:08,033 INFO    AMR: 0.0018779342723004694
2022-02-21 00:41:08,035 INFO    Selected train apps: ['CoMD']
2022-02-21 00:41:09,388 INFO    
               precision    recall  f1-score   support

        none       0.94      0.25      0.39     10360
       dcopy       0.65      0.40      0.50       265
    memeater       0.03      0.69      0.05       270
        leak       1.00      0.50      0.66       265
        dial       0.05      0.23      0.09       265

    accuracy                           0.27     11425
   macro avg       0.53      0.41      0.34     11425
weighted avg       0.89      0.27      0.39     11425

202

2022-02-21 00:41:21,486 INFO    train:miniAMR#test:cg-mg-lu-miniGhost-miniMD-CoMD#model:RandomForestClassifier#cv:0
2022-02-21 00:41:21,505 INFO    Total misclassified normal runs: 3506, Total normal runs 10360 
2022-02-21 00:41:21,505 INFO    FAR: 0.3384169884169884
2022-02-21 00:41:21,506 INFO    Total misclassified anom runs: 3, Total anom runs 1065 
2022-02-21 00:41:21,507 INFO    AMR: 0.0028169014084507044
2022-02-21 00:41:21,509 INFO    Selected train apps: ['sp', 'kripke', 'bt']
2022-02-21 00:41:21,510 INFO    File exists
2022-02-21 00:41:21,510 INFO    Selected train apps: ['sp', 'kripke', 'ft']
2022-02-21 00:41:21,511 INFO    File exists
2022-02-21 00:41:21,511 INFO    Selected train apps: ['sp', 'kripke', 'miniAMR']
2022-02-21 00:41:21,512 INFO    File exists
2022-02-21 00:41:21,512 INFO    Selected train apps: ['sp', 'bt', 'ft']
2022-02-21 00:41:21,513 INFO    File exists
2022-02-21 00:41:21,513 INFO    Selected train apps: ['sp', 'bt', 'miniAMR']
2022-02-21 00:41:21,514 INF

2022-02-21 00:41:31,405 INFO    File exists


### 5-fold CV Validation

In [29]:
SCALER = 'MinMax'

if SCALER == 'MinMax':
    
    scaler = MinMaxScaler()    

elif SCALER == 'Standard':
    
    scaler = StandardScaler()

clf = RandomForestClassifier(random_state=0)

pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])

In [31]:
scoring = ['precision_macro', 'recall_macro', 'f1_macro', 'f1_weighted']
skf = StratifiedKFold(n_splits=5)

scores = cross_validate(pipeline, all_data, all_label['anom'].values,                         
                        cv=skf, 
                        scoring=scoring)

In [32]:
scores

{'fit_time': array([37.87093496, 33.30407429, 31.670362  , 32.61878467, 33.78410292]),
 'score_time': array([0.22262263, 0.16374779, 0.16079235, 0.16500545, 0.16272569]),
 'test_precision_macro': array([0.99701681, 0.9971924 , 0.99916055, 0.99936975, 0.99889966]),
 'test_recall_macro': array([0.97295918, 0.96869731, 0.96666667, 0.975     , 0.95625   ]),
 'test_f1_macro': array([0.98429409, 0.9816149 , 0.98139757, 0.98635104, 0.97488691]),
 'test_f1_weighted': array([0.9967994 , 0.99602717, 0.99600515, 0.99703788, 0.99467834])}